In [8]:
import dash
from dash import Dash, html, Input, Output, ctx, callback
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import plotly.graph_objects as go
from flask_ngrok import run_with_ngrok

# Cargar los datos
file_path = '6 class csv.csv'
data = pd.read_csv(file_path)

# Preparar los datos para el entrenamiento
X = data[['Temperature (K)', 'Luminosity(L/Lo)', 'Radius(R/Ro)', 'Absolute magnitude(Mv)']]
y = data['Star type']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarizar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar el modelo SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_scaled, y_train)

# Evaluar el modelo
y_pred_train = svm_model.predict(X_train_scaled)
y_pred_test = svm_model.predict(X_test_scaled)
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Crear la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
run_with_ngrok(app)  # Inicia ngrok cuando se ejecute el servidor

app.layout = dbc.Container([
    dbc.Tabs([
        dbc.Tab(label='Clasificación de Estrellas', tab_id='tab-1'),
        dbc.Tab(label='Información del Modelo', tab_id='tab-2')
    ], id='tabs', active_tab='tab-1'),
    html.Div(id='content')
])

@app.callback(
    Output('content', 'children'),
    Input('tabs', 'active_tab')
)
def render_content(tab):
    if tab == 'tab-1':
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H1("Clasificación de Estrellas", className='text-center my-4'),
                    dbc.Card(
                        dbc.CardBody([
                            html.Label("Temperatura (K):"),
                            dcc.Input(id='input-temp', type='number', value=5000, step=100, min=0, max=40000, className='form-control mb-2'),
                            html.Br(),
                            html.Label("Luminosidad (L/Lo):"),
                            dcc.Input(id='input-lum', type='number', value=1, step=0.1, className='form-control mb-2'),
                            html.Label("Radio (R/Ro):"),
                            dcc.Input(id='input-radius', type='number', value=1, step=0.1, className='form-control mb-2'),
                            html.Label("Magnitud Absoluta (Mv):"),
                            dcc.Input(id='input-mag', type='number', value=5, step=0.1, className='form-control mb-2'),
                            html.Br(),
                            dbc.Button("Clasificar", id='classify-button', color='primary', className='mt-2'),
                            html.Hr(),
                            #Agregamos botones
                            html.Label("Escoge el tipo de estrella que deseas ver:"),
                            html.Button('0', id='btn-nclicks-0', n_clicks=0),
                            html.Button('1', id='btn-nclicks-1', n_clicks=0),
                            html.Button('2', id='btn-nclicks-2', n_clicks=0),
                            html.Button('3', id='btn-nclicks-3', n_clicks=0),
                            html.Button('4', id='btn-nclicks-4', n_clicks=0),
                            html.Button('5', id='btn-nclicks-5', n_clicks=0),
                            html.Div(id='container-button-timestamp'),
                            html.Div(id='output-class', className='text-center mt-4')
                        ]),
                    )
                ], width=4),
                dbc.Col([
                    html.H3("Gráfico de Dispersión de Estrellas", className='text-center my-4'),
                    dcc.Graph(id='star-scatter', figure={})
                ], width=8),
                
            ], className='mt-4')
        ])
    elif tab == 'tab-2':
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H5("Modelo Utilizado: Support Vector Machine (SVM)"),
                    html.P("El modelo SVM con kernel lineal se utiliza para clasificación."),
                    html.P("El modelo fue elegido por su efectividad en problemas de clasificación con datos de alta dimensionalidad."),
                    html.P(f"Precisión del conjunto de entrenamiento: {train_accuracy:.2%}"),
                    html.P(f"Precisión del conjunto de prueba: {test_accuracy:.2%}")
                ], width=12)
            ], className='mt-4'),
            dbc.Row([
                dbc.Col([
                    dcc.Graph(
                        id='accuracy-graph',
                        figure=go.Figure(
                            data=[go.Bar(x=['Train Accuracy', 'Test Accuracy'], y=[train_accuracy, test_accuracy], text=[f'{train_accuracy:.2%}', f'{test_accuracy:.2%}'], textposition='auto')],
                            layout_title_text='Precisión del Modelo'
                        )
                    )
                ], width=12)
            ], className='mt-4')
        ])




@app.callback(
    Output('output-class', 'children'),
    Output('star-scatter', 'figure'),
    Input('classify-button', 'n_clicks'),
    State('input-temp', 'value'),
    State('input-lum', 'value'),
    State('input-radius', 'value'),
    State('input-mag', 'value')
)
def classify_star(n_clicks, temp, lum, radius, mag):
    if n_clicks is None:
        return "", {}

    # Preprocesar los valores de entrada
    input_data = pd.DataFrame([[temp, lum, radius, mag]], columns=['Temperature (K)', 'Luminosity(L/Lo)', 'Radius(R/Ro)', 'Absolute magnitude(Mv)'])
    input_scaled = scaler.transform(input_data)

    # Realizar la predicción
    star_type = svm_model.predict(input_scaled)[0]

    # Crear la anotación para la estrella consultada
    annotation = {
        'x': temp,
        'y': lum,
        'xref': 'x',
        'yref': 'y',
        'text': f"Estrella Consultada<br>Tipo: {star_type}",
        'showarrow': True,
        'arrowhead': 2,
        'ax': 0,
        'ay': -40
    }

    # Actualizar el gráfico de dispersión
    fig = px.scatter(data, x='Temperature (K)', y='Luminosity(L/Lo)', color='Star type',
                     title='Clasificación de Estrellas en la Secuencia Principal',
                     labels={'Temperature (K)': 'Temperatura (K)', 'Luminosity(L/Lo)': 'Luminosidad (L/Lo)'},
                     hover_data=['Radius(R/Ro)', 'Absolute magnitude(Mv)', 'Star color', 'Spectral Class'])
    

    fig.update_layout(
        transition_duration=500,
        annotations=[annotation],
        xaxis_autorange='reversed',  # Invertir el eje x
        yaxis_type='log'  # Escala logarítmica para el eje y
    )
    return dbc.Alert(f"Tipo de estrella predicho: {star_type}", color='info'), fig


#callback a los botones
@app.callback(
    Output('container-button-timestamp', 'children'),
    Output('star-scatter', 'figure'),
    Input('btn-nclicks-0', 'n_clicks'),
    Input('btn-nclicks-1', 'n_clicks'),
    Input('btn-nclicks-2', 'n_clicks'),
    Input('btn-nclicks-3', 'n_clicks'),
    Input('btn-nclicks-4', 'n_clicks'),
    Input('btn-nclicks-5', 'n_clicks')
)


def displayClick(btn0, btn1, btn2, btn3, btn4, btn5):

    if "btn-nclicks-0" == ctx.triggered_id:
        estrella = 0
    elif "btn-nclicks-1" == ctx.triggered_id:
        estrella = 1
    elif "btn-nclicks-2" == ctx.triggered_id:
        estrella = 2
    elif "btn-nclicks-3" == ctx.triggered_id:
        estrella = 3
    elif "btn-nclicks-4" == ctx.triggered_id:
        estrella = 4
    elif "btn-nclicks-5" == ctx.triggered_id:
        estrella = 5
# tenemos la grafica que solo selecciona un tipo de estrella

    fig = px.scatter(data[data['Star type'] == estrella], x='Temperature (K)', y='Luminosity(L/Lo)', color='Star type',
                     title='Clasificación de Estrellas en la Secuencia Principal',
                     labels={'Temperature (K)': 'Temperatura (K)', 'Luminosity(L/Lo)': 'Luminosidad (L/Lo)'},
                     hover_data=['Radius(R/Ro)', 'Absolute magnitude(Mv)', 'Star color', 'Spectral Class'])
    

    fig.update_layout(
        transition_duration=500,
        annotations=[annotation],
        xaxis_autorange='reversed',  # Invertir el eje x
        yaxis_type='log'  # Escala logarítmica para el eje y
    )

    return fig

    





# Índice de tipos de estrellas
index_content = dbc.Container([
    html.H5("Índice de Tipos de Estrellas"),
    html.P("Brown Dwarf -> Star Type = 0"),
    html.P("Red Dwarf -> Star Type = 1"),
    html.P("White Dwarf -> Star Type = 2"),
    html.P("Main Sequence -> Star Type = 3"),
    html.P("Supergiant -> Star Type = 4"),
    html.P("Hypergiant -> Star Type = 5")
], className='mt-4')

# Agregar el índice al layout de la aplicación
app.layout.children.append(index_content)

# Ejecutar la aplicación
app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
Exception in thread Thread-87:
Traceback (most recent call last):
  File "d:\U\Python\lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
  File "d:\U\Python\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "d:\U\Python\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "d:\U\Python\lib\site-packages\urllib3\connectionpool.py", line 793, in urlopen
    response = self._make_request(
  File "d:\U\Python\lib\site-packages\urllib3\connectionpool.py", line 496, in _make_request
    conn.request(
  File "d:\U\Python\lib\site-packages\urlli

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "d:\U\Python\lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\U\Python\lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\U\Python\lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "d:\U\Python\lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "d:\U\Python\lib\site-packages\dash\dash.py", line 1273, in dispatch
    ctx.run(
  File "d:\U\Python\lib\site-packages\dash\_callback.py", line 440, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
  File "C:\Users\vliza\AppData\Local\Temp\ipykernel_14304\1331575113.py", line 66, in render_content
    dcc.Inpu

127.0.0.1 - - [03/Jul/2024 15:45:56] "POST /_dash-update-component HTTP/1.1" 500 -


In [ ]:
file_path = '6 class csv.csv'
data = pd.read_csv(file_path)
px.scatter(data[data['Star type'] == 5], x='Temperature (K)', y='Luminosity(L/Lo)', color='Star type',
                     title='Clasificación de Estrellas en la Secuencia Principal',
                     labels={'Temperature (K)': 'Temperatura (K)', 'Luminosity(L/Lo)': 'Luminosidad (L/Lo)'},
                     hover_data=['Radius(R/Ro)', 'Absolute magnitude(Mv)', 'Star color', 'Spectral Class'])

In [ ]:
data = pd.read_csv(file_path)
y = data[data['Star type'] == 1],[data['Star type'] == 2]
y

(     Temperature (K)  Luminosity(L/Lo)  Radius(R/Ro)  Absolute magnitude(Mv)  \
 10              3600           0.00290        0.5100                  10.690   
 11              3129           0.01220        0.3761                  11.790   
 12              3134           0.00040        0.1960                  13.210   
 13              3628           0.00550        0.3930                  10.480   
 14              2650           0.00060        0.1400                  11.782   
 15              3340           0.00380        0.2400                  13.070   
 16              2799           0.00180        0.1600                  14.790   
 17              3692           0.00367        0.4700                  10.800   
 18              3192           0.00362        0.1967                  13.530   
 19              3441           0.03900        0.3510                  11.180   
 70              3345           0.02100        0.2730                  12.300   
 71              3607       